# React Prompting without similar Problems

In [ ]:
import json
from functions import inference
from new_templates import *
from templates import *

In [ ]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from typing import Annotated
from langchain_experimental.tools import PythonREPLTool

In [ ]:
python_repl_tool = PythonREPLTool()

In [ ]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    """Creating an agent"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [ ]:
def agent_node(state, agent, name, log):
    result = agent.invoke(state)
    log[name] = result["output"]
    return {"messages": [HumanMessage(content=result['output'], name=name)]}

In [ ]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Noter", "Validator", "Explainer"]
system_prompt = (
    "你是一名监督员，负责管理以下成员之间的对话：{members}。"
    "根据以下用户请求，指定下一个需要行动的成员。每个成员将执行一个任务,"
    "你的任务是安排Noter, Explainer, Validator之间的工作，让他们合作写出一个完美的题目解析"
    "Validator检查Noter的板书是否正确，如果不正确应该将正确答案返回给Noter让其根据正确答案重新生成符合要求的板书"
    "当Noter重新生成板书后，请将Noter的板书再次传递给Validator查验，只有当Validator的验证Noter板书的正确性后，转去Explainer"
    "每一步反馈他们的结果和状态。任务完成后，请回复“FINISH”。"
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "选择下一步由谁来执行",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "根据以上内容，下一步应该轮到谁了？"
            "或者我们是否已经得到不错的讲解，可以“完成”了呢？"
            "请从以下选项中选一个：{options}",
        )
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [ ]:
import functools
import operator
from typing import Sequence, TypedDict

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str
log = {}

noter_agent = create_agent(llm, [python_repl_tool], noter_template)
noter_node = functools.partial(agent_node, agent=noter_agent, name="Noter", log=log)

explainer_agent = create_agent(llm, [python_repl_tool], explainer_react_template)
explainer_node = functools.partial(agent_node, agent=explainer_agent, name="Explainer", log=log)

validator_agent = create_agent(llm, [python_repl_tool], validator_correctness_template)
validator_node = functools.partial(agent_node, agent=validator_agent, name="Validator", log=log)

def supervisor_node(state, log):
    result = supervisor_chain.invoke(state)
    log["Supervisor"] = result
    return result

workflow = StateGraph(AgentState)
workflow.add_node("Noter", noter_node)
workflow.add_node("Explainer", explainer_node)
workflow.add_node("Validator", validator_node)
workflow.add_node("Supervisor", functools.partial(supervisor_node, log=log))

In [ ]:
for member in members:
    workflow.add_edge(member, "Supervisor")

conditional_map = {k:k for k in members}
conditional_map['FINISH'] = END
workflow.add_conditional_edges("Supervisor", lambda x: x['next'], conditional_map)
workflow.add_edge(START, "Supervisor")

graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    pass

In [ ]:
import json
with open("test_problem_set.json", 'r') as file:
    problem_set = json.load(file)
problems = []
answers = []
for i in range(len(problem_set)):
    problems.append(problem_set[i]['problem'])
    answers.append(problem_set[i]['answer'])

In [ ]:
log = []
for i in range(100):
    record = []
    problem = "### 原题：" + problems[i]
    answer = "### 标准答案：" + answers[i]
    try:
        for s in graph.stream(
            {
                "messages": [
                    HumanMessage(content=problem),
                    HumanMessage(content=answer)
                ],
            },
            {"recursion_limit": 10},
        ):
            if "__end__" not in s:
                record.append(s)
                print(s)
                print("----")
        log.append(record)
    except Exception as e:
        log.append(["error", "error"])


In [18]:
# temp module
with open("outputs/model_2_output.json", 'r') as file:
    data = json.load(file)

for i in range(100):
    if 'Error' in data[i]['Note']:
        record = []
        problem = "### 原题：" + problems[i]
        answer = "### 标准答案：" + answers[i]
        try:
            for s in graph.stream(
                {
                    "messages": [
                        HumanMessage(content=problem),
                        HumanMessage(content=answer)
                    ],
                },
                {"recursion_limit": 10},
            ):
                if "__end__" not in s:
                    record.append(s)
                    print(s)
                    print("----")
                    for item in record:
                        for role in item:
                            if role == "Supervisor" or role == "Validator": continue
                            if role == "Noter":
                                data[i]['Note'] = item[role]['messages'][0].content
                            if role == "Explainer":
                                data[i]['Expl'] = item[role]['messages'][0].content


        except Exception as e:
            log.append(["error", "error"])

{'Supervisor': {'next': 'Noter'}}
----
{'Noter': {'messages': [HumanMessage(content='知识点讲解: 本题考查的知识点是方程的阶数与一次方程的概念，解题关键点是确定方程中的指数使得方程成为一次方程。\n\n### 板书一:\n依题意得\n\n\\(3x^m + 2y^n = 4\\)\n\n因为方程是二元一次方程，所以每一项的阶数为1\n\n故 \\(m = 1\\)\n\n由于 \\(3x^m\\) 中 \\(3x\\)的系数是3,且需要满足一次方程的条件\n\n所以 \\(m = 1\\)\n\n### 板书二:\n故 \\(m = \\frac{1}{3}\\)\n\n再考虑 \\(2y^n\\) 中 \\(2y\\)的系数是2,且需要满足一次方程的条件\n\n所以 \\(n = 1\\)\n\n故答案是 \\(m = \\frac{1}{3} , n = 1\\)', name='Noter')]}}
----
{'Supervisor': {'next': 'Validator'}}
----
{'Validator': {'messages': [HumanMessage(content='### 验证过程：\n\n1. **理解题意**：\n   - 题目要求方程 \\(3x^m + 2y^n = 4\\) 是关于 \\(x\\) 和 \\(y\\) 的二元一次方程。\n   - 二元一次方程的定义是方程中变量的最高次幂分别为1。\n\n2. **验证Noter的解答**：\n   - Noter的答案是 \\(m = \\frac{1}{3}\\) 和 \\(n = 1\\)。\n\n3. **检查Noter步骤**：\n   - 板书一中，Noter首先认为因为是一次方程，故 \\(m = 1\\)。但随后纠正为 \\(m = \\frac{1}{3}\\)，并没有给出明确的逻辑。\n   - 正确的逻辑应该是：\n     - 对于 \\(3x^m\\)，为了使这个项变成一次方程的一部分，\\(x\\) 的指数 \\(m\\) 必须为1。\n     - 对于 \\(2y^n\\)，为了使这个项变成一次方程的一部分，\\(y\\) 的指数 \\(n\\) 也必须为1

In [20]:
data[36]

{'problem': '某校计划购买《论语》和《孟子》供学生阅读，已知用1300元购买了《孟子》和《论语》各20本，《孟子》的单价比《论语》的单价少15元。\n(1)《孟子》的单价是______元，《论语》的单价是______元；\n(2)根据需要，学校决定再次购买两种书共50本，正逢书店开展“优惠促销”活动，《孟子》单价优惠4元，《论语》的单价打8折。如果此次学校购买书的总费用不超过1500元，且购买《论语》不少于38本，则有哪几种购买方案？为了节约资金，学校应该选择哪种方案？为什么？',
 'Note': '知识点讲解: 本题考查的知识点是方程组的解法与简单的成本计算，解题关键点是根据已知条件列出方程组求解单价，并通过计算不同方案的总费用来选择最优方案。\n\n### 板书一:\n(1)\n20x + 20y = 1300\n\nx = y - 15\n\n解得:\nx = 25\ny = 40\n\n### 板书二:\n(2)\n由题意，得《孟子》的单价是:\n25 - 4 = 21 (元)\n\n《论语》的单价是:\n40 × 0.8 = 32 (元)\n\n方案1:\n32 × 38 + 21 × 12 = 1468 (元)\n\n方案2:\n32 × 39 + 21 × 11 = 1479 (元)\n\n方案3:\n32 × 40 + 21 × 10 = 1490 (元)\n\n1468 < 1479 < 1490\n\n故答案是: 学校应选择方案1：购买《论语》38本，《孟子》12本。',
 'Expl': '### 讲解一：\n我们首先来看题目提供的条件：某校用1300元购买了《孟子》和《论语》各20本，并且《孟子》的单价比《论语》的单价少15元。\n\n1. 请同学们思考一下，这里我们可以设《孟子》的单价为x元，《论语》的单价为y元。\n2. 这说明了什么?\n   - 我们可以得到两个方程：\n     - 第一个方程表示两种书的总费用：20x + 20y = 1300。\n     - 第二个方程表示两者的价格差异：x = y - 15。\n3. 那么接下来我们应该做什么？\n   - 我们需要利用这两个方程来求解x和y的值。\n\n### 讲解二：\n接下来，我们将第二个方程x = y - 15代入第一个方程20x + 20y = 1300

In [21]:
with open('model_2_output.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
final_notes = []
final_expls = []
for i in range(len(log)):
    noter_log = []
    explainer_log = []
    # print(log[i])
    record = log[i]
    if record[0] == "error" and record[1] == "error":
        final_notes.append("Error")
        final_expls.append("Error")
        continue
    for item in record:
        for role in item:
            if role == "Supervisor" or role == "Validator": continue
            if role == "Noter":
                noter_log.append(item[role]['messages'][0].content)
            if role == "Explainer":
                explainer_log.append(item[role]['messages'][0].content)
    final_notes.append(noter_log[-1])
    final_expls.append(explainer_log[-1])

In [ ]:
pbl_set = []
for i in range(len(final_notes)):
    cur = {}
    cur['problem'] = problems[i]
    cur['Note'] = final_notes[i]
    cur['Expl'] = final_expls[i]
    pbl_set.append(cur)

with open("outputs/model_2_ouptut.json", 'w', encoding='utf-8') as file:
    json.dump(pbl_set, file, ensure_ascii=False, indent=4)

In [ ]:
print(final_expls[0])

In [ ]:
for problem in problems[:1]:
    cur_note = await inference({"problem": problem}, noter_template)
    react_expl = await inference({"note": cur_note}, explainer_react_template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
async def check(note1: str, note2: str) -> bool:
    llm = ChatOpenAI(model='gpt-4o')
    sys_prompt =    r"""
                        你是一个题解检查者，你负责检查两个题解的最终答案是否一致。注意：题解不需要一模一样，
                        只要保证最后每一问的答案都完全一致即可。
                        如果答案一致：请输出是。如果答案不一致：请输出否。请确保输出一个字，不要输出“是“与”否”意外的内容

                        题解1:
                        {note1}
                        
                        题解2:
                        {note2}
                    """
    prompt = ChatPromptTemplate.from_template(sys_prompt)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    response = chain.invoke(input={"note1": note1, "note2": note2})
    return response

In [ ]:
with open("output_with_answer.json", 'r') as file:
    data = json.load(file)

score = 0
is_same = [0 for _ in range(len(data))]
for i in range(len(data)):
    print(f"----- processing {i} -----")
    response = await check(data[i]['Note'], answers[i])
    if response == "是":
        score += 1
        is_same[i] = 1

score /= len(data)
score

In [ ]:
for i in range(len(is_same)):
    if is_same[i] == 0: print(i)
    

In [ ]:
import json
with open("output_with_answer.json", 'r') as file:
    data = json.load(file)

In [ ]:
problems[35]

In [ ]:
answers[35]

In [ ]:
data[35]